In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

$$
L(\theta\mid \textbf{x}) = \prod_{i=1}^n f(x_i;\theta) = \prod_{i=1}^n {\lambda^{X_i} e^{-\lambda} \over X_i !}
= \lambda^n e^{-\lambda \sum x_i}
$$

In [13]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])
len(my_data)

20

In [30]:
ML_lam=1/np.mean(my_data)
print('Our Max Likelihood lambda is:', ML_lam)

hyp_lam=3 #what we are testing
print('We are testing this agianst our hypothesis lambda=', hyp_lam)
n=len(my_data) #trials
    
hyp_like=hyp_lam**(n)*e**(-1*hyp_lam*sum(my_data))

ML_like=ML_lam**(n)*e**(-1*ML_lam*sum(my_data))

LRTS=hyp_like/ML_like
chi1=-2*log(LRTS)
print('LRTS:', LRTS, 'Chi(1) value:', chi1)
crit=stats.chi2.ppf(.95, 1)
print('Using a 5% confidence, our critical chi squared value is:', crit)
print('Our LRS p-value is:', stats.chi2.cdf(4.719, 1))

Our Max Likelihood lambda is: 5.089058524173028
We are testing this agianst our hypothesis lambda= 3
LRTS: 0.0944569427967816 Chi(1) value: 4.719222360188458
Using a 5% confidence, our critical chi squared value is: 3.841458820694124
Our LRS p-value is: 0.9701689134021486


Thus we reject the null hypothesis that $\lambda=3$, because our test chi squared valueof 4.719 falls outside of our 95% confidence chi squared value of 3.84.

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [7]:
crit=stats.chi2.ppf(.95, 1)
crit

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [12]:
n=20
lam0=3
ts=[]
for _ in np.arange(10000):
    my_data=stats.expon.rvs(size=n, scale= 1/5)
    test_stat=-2*log((np.mean(my_data)**n)*(lam0**n)*e**(-lam0*sum(my_data)+n))
    ts=np.append(ts, test_stat)
np.mean(ts)

5.443829426869639

Repeat for a sample size of 50. What do you expect to happen to power? 

In [11]:
n=50
lam0=3
ts=[]
for _ in np.arange(10000):
    my_data=stats.expon.rvs(size=n, scale= 1/5)
    test_stat=-2*log((np.mean(my_data)**n)*(lam0**n)*e**(-lam0*sum(my_data)+n))
    ts=np.append(ts, test_stat)
np.mean(ts)

12.10003418365914

We expect the power to increase as we increase our sample size. This is exactly what happens. Keeping everything constant, but increasing the sample size, our mean test statistic rises, meaning it is more probable that there will not be a false "fail to reject" on the null hypothesis.

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

My test statistic will be the Max Likelihood lambda value from a sample without replacement from my_data.
<dv>
If lambda is equal to 3, we should expect only about 50% of the data to be above or below it.

In [28]:
n=20
ML_lams=[]
for i in np.arange(10000):
    ts=1/(np.mean(np.random.choice(my_data, n, replace=True)))
    ML_lams=np.append(ML_lams, ts)
if np.mean(ML_lams)>=3:
    p_val=sum(ML_lams>3)/10000
else:
    p_val=sum(ML_lams<3)/10000
p_val

0.9994

How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

This p-value was much closer to 1 than our LRT p-value, even though both of them reject the null hypothesis.

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [141]:
hyp_true_data=[]
for i in np.arange(10000):
    ts=1/np.mean(stats.expon.rvs(scale=1/3, size=20))
    hyp_true_data=np.append(hyp_true_data, ts)

In [142]:
upper_reject=4.5
lower_reject=2.17
upper_val=sum((hyp_true_data)>upper_reject)/10000
lower_val=sum((hyp_true_data)<lower_reject)/10000
print(upper_val, lower_val)
print('We will reject outside of the interval of: (', lower_reject, ',', upper_reject, ')')

0.0522 0.0577
We will reject outside of the interval of: ( 2.17 , 4.5 )


Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [143]:
n=20
ML_lams=[]
reject=[]
for i in np.arange(10000):
    ts=1/(np.mean(np.random.choice(my_data, n, replace=True)))
    ML_lams=np.append(ML_lams, ts)
    if (upper_reject < ts) or (lower_reject > ts):
        reject=np.append(reject, True)
    else:
        reject=np.append(reject, False)
sum(reject)/10000

0.7724

Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [144]:
hyp_true_data=[]
for i in np.arange(10000):
    ts=1/np.mean(stats.expon.rvs(scale=1/3, size=50))
    hyp_true_data=np.append(hyp_true_data, ts)

In [145]:
upper_reject_50=3.86
lower_reject_50=2.43
upper_val=sum((hyp_true_data)>upper_reject_50)/10000
lower_val=sum((hyp_true_data)<lower_reject_50)/10000
print(upper_val, lower_val)
print('We will reject outside of the interval of: (', lower_reject_50, ',', upper_reject_50, ')')

0.0523 0.0588
We will reject outside of the interval of: ( 2.43 , 3.86 )


In [146]:
n=50
ML_lams=[]
reject=[]
for i in np.arange(10000):
    ts=1/(np.mean(np.random.choice(my_data, n, replace=True)))
    ML_lams=np.append(ML_lams, ts)
    if (upper_reject_50 < ts) or (lower_reject_50 > ts):
        reject=np.append(reject, True)
    else:
        reject=np.append(reject, False)
sum(reject)/10000

0.9949

It is very clear to see that we reject many more of the test statistics when the sample size increases.